In [1]:
# torch imports
import torch
# Misc imports
import os 
import sys
import seaborn as sns
from pathlib import Path
sys.path.append('/storage/vbutoi/projects')
sys.path.append('/storage/vbutoi/libraries')
sys.path.append('/storage/vbutoi/projects/ESE')
sys.path.append('/storage/vbutoi/projects/UniverSegDev')
# Ionpy imports
from ionpy.util import Config
from ionpy.analysis import ResultsLoader

# Define some useful paths.
os.environ['DATAPATH'] = ':'.join((
       '/storage/vbutoi/datasets',
       '/storage'
))
# Set some defaults
rs = ResultsLoader()
sns.set_style("darkgrid")
torch.set_printoptions(linewidth=200)

# For using code without restarting.
%load_ext autoreload
%autoreload 2
# For using yaml configs.
%load_ext yamlmagic

In [2]:
%%yaml default_cfg 

experiment:
    exp_root: '?'
    inference_seed: 40
    pred_threshold: 0.5 

log:
    root: '?'
    save_preds: False 
    log_interval: 10 
    log_pixel_stats: False 
    gether_inference_stats: True
    compute_global_metrics: False 

dataloader:
    batch_size: '?'
    num_workers: 1 
    pin_memory: True 

<IPython.core.display.Javascript object>

In [3]:
%%yaml calibration_cfg 

calibrator:
    _name: None 

local_calibration:
    num_prob_bins: 15
    neighborhood_width: 3

global_calibration:
    num_classes: '?' 
    num_prob_bins: 15
    neighborhood_width: 3

<IPython.core.display.Javascript object>

In [4]:
%%yaml model_cfg 

# For standard datasets
#####################################
model:
    pred_label: '?' 
    _type: "standard"
    pretrained_exp_root : None
    checkpoint: 'min-val-area_estimation_error'
    # checkpoint: 'min-val-ece_loss'

<IPython.core.display.Javascript object>

## Gather Inference Options.

In [5]:
%%yaml experiment_cfg 

# Use this field if you want to call it something else beyond the base_model_dir.
name: "OCTA_FR_All_Models_wPreds"

base_model: 
    - "/storage/vbutoi/scratch/ESE/training/08_24_24_HeptaticVessel_LowerLR"
    - "/storage/vbutoi/scratch/ESE/calibration/08_08_24_OCTA_FULLRES_TempScaling_CHECKPOINTS"
    - "/storage/vbutoi/scratch/ESE/calibration/08_11_24_OCTA_FULLRES_LTS_LongRuns"

# Optimal thresholds for OCTA
# experiment:
#     pred_threshold:
#         - 0.184
#         - 0.225
#         - 0.234
#         - 0.25  
#         - 0.275 
#         - 0.325

inference_data:
    _class: "ese.datasets.OCTA_6M"
    # split: "val"
    split: ("cal", "val") # We optimize thresholds on the calibration set, but evaluate on the validation set.
    version: 1.0 # For full res, 1.0 is the only version.
    label_threshold: null # Need to compare with clean gt.

# We want to test if we can save preds.
log:
    save_preds: True

dataloader:
    batch_size: 8

<IPython.core.display.Javascript object>

In [6]:
# Local imports
from ese.analysis.analysis_utils.submit_utils import get_ese_inference_configs

# Get the configs for the different runs.
base_cfg = Config(default_cfg).update([calibration_cfg, model_cfg])
# Get the different experiment cfg yamls.
inf_cfgs = get_ese_inference_configs(
    exp_cfg=experiment_cfg, 
    base_cfg=base_cfg
)

In [7]:
len(inf_cfgs)

64

## Running Jobs

In [8]:
from ese.analysis.run_inference import get_cal_stats

/local/vbutoi/envs/UniverSegTF/lib/python3.9/site-packages/pydantic/_internal/_fields.py:149: UserWarning: Field "model_outputs" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/storage/vbutoi/libraries/pylot/pandas/register.py:47: UserWarning: registration of accessor <class 'pylot.pandas.register.register_series_method.<locals>.inner.<locals>.AccessorMethod'> under name 'fillNA' for type <class 'pandas.core.series.Series'> is overriding a preexisting attribute with the same name.
  register_series_accessor(method.__name__)(AccessorMethod)
/storage/vbutoi/libraries/pylot/pandas/register.py:47: UserWarning: registration of accessor <class 'pylot.pandas.register.register_series_method.<locals>.inner.<locals>.AccessorMethod'> under name 'isNA' for type <class 'pandas.core.series.Series'> is overriding a preexisting attribute with the same name.
  register_series_accessor(meth

In [9]:
# from ese.experiment import run_ese_exp

# ###### Run individual jobs
# run_ese_exp(
#     config=inf_cfgs[0], 
#     job_func=get_cal_stats,
#     run_name='debug',
#     # show_examples=True,
#     show_examples=False,
#     gpu='0',
#     # gpu='4',
# ) 

In [10]:
from ese.experiment import submit_ese_exps 

#### Run Batch Jobs
submit_ese_exps(
    config_list=inf_cfgs,
    job_func=get_cal_stats,
    available_gpus=['0', '1', '2', '3']
    # available_gpus=['4', '5', '6', '7']
)

Submitting job 1/64:
--> Launched job-id: 920055 on gpu: 0.
Submitting job 2/64:
--> Launched job-id: 920171 on gpu: 1.
Submitting job 3/64:
--> Launched job-id: 920343 on gpu: 2.
Submitting job 4/64:
--> Launched job-id: 920515 on gpu: 3.
Submitting job 5/64:
--> Launched job-id: 920740 on gpu: 0.
Submitting job 6/64:
--> Launched job-id: 920947 on gpu: 1.
Submitting job 7/64:
--> Launched job-id: 921179 on gpu: 2.
Submitting job 8/64:
--> Launched job-id: 921392 on gpu: 3.
Submitting job 9/64:
--> Launched job-id: 921602 on gpu: 0.
Submitting job 10/64:
--> Launched job-id: 921811 on gpu: 1.
Submitting job 11/64:
--> Launched job-id: 922018 on gpu: 2.
Submitting job 12/64:
--> Launched job-id: 922226 on gpu: 3.
Submitting job 13/64:
--> Launched job-id: 922433 on gpu: 0.
Submitting job 14/64:
--> Launched job-id: 922640 on gpu: 1.
Submitting job 15/64:
--> Launched job-id: 922848 on gpu: 2.
Submitting job 16/64:
--> Launched job-id: 923056 on gpu: 3.
Submitting job 17/64:
--> Launche